In [1]:
import sys

sys.path.append('..')

from action_language_interpreter.action_language import *

In [2]:
AD = ActionDescription()

In [3]:
A = FluentLiteral('A')
B = FluentLiteral('B')
C = FluentLiteral('C')
D = FluentLiteral('D')
E = FluentLiteral('E')
F = FluentLiteral('F')

In [4]:
AD.add_stmt_impossible_if('e1', A)
AD.add_stmt_causes_if('e1', E, ~E)
AD.add_stmt_causes_if('e2', D, ~D)
AD.add_stmt_causes_if('e3', A, ~A)
AD.add_stmt_causes_if('e3', C, ~C)
AD.add_stmt_impossible_if('e3', ~E)
AD.add_stmt_impossible_if('e3', ~F)
AD.add_stmt_if(B, C)
AD

({e3 causes C if ¬C, e1 causes E if ¬E, e2 causes D if ¬D, e3 causes A if ¬A}, {B if C}, {e1 impossible_if A, e3 impossible_if ¬E, e3 impossible_if ¬F})

In [5]:
events = {'e1', 'e2', 'e3'}
events

{'e1', 'e2', 'e3'}

In [6]:
state1 = {~A, ~B, ~C, ~D, ~E, F}
state1

{¬A, ¬B, ¬C, ¬D, ¬E, F}

## Action 1

In [7]:
AD.executable_events(state1, events)

(frozenset({'e1', 'e2'}), (e3 impossible_if ¬E,))

In [8]:
state2,relevant_statements_1 = AD.execute(state1, 'e1')

In [9]:
state2

{¬A, ¬B, ¬C, ¬D, E, F}

In [10]:
relevant_statements_1

[e1 causes E if ¬E]

## Action 2

In [11]:
AD.executable_events(state2, events)

(frozenset({'e1', 'e2', 'e3'}), ())

In [12]:
state3, relevant_statements_2 = AD.execute(state2, 'e2')

In [13]:
state3

{¬A, ¬B, ¬C, D, E, F}

In [14]:
relevant_statements_2

[e2 causes D if ¬D]

## Action 3

In [15]:
AD.executable_events(state3, events)

(frozenset({'e1', 'e2', 'e3'}), ())

In [16]:
state4, relevant_statements_3 = AD.execute(state3, 'e3')

In [17]:
state4

{A, B, C, D, E, F}

In [18]:
relevant_statements_3

[e3 causes C if ¬C, e3 causes A if ¬A, B if C]

## Scenario Path

In [19]:
scenario_path:ScenarioPath = (state1, [('e1', state2), ('e2', state3), ('e3', state4)])
scenario_path

({¬A, ¬B, ¬C, ¬D, ¬E, F},
 [('e1', {¬A, ¬B, ¬C, ¬D, E, F}),
  ('e2', {¬A, ¬B, ¬C, D, E, F}),
  ('e3', {A, B, C, D, E, F})])

## Transition Event

In [20]:
transition_events = get_transition_events(scenario_path, state4)
transition_events

{'e3'}

## First Causal Explanation

In [21]:
direct_effects,indirect_effects = get_first_causal_explanation(AD, scenario_path, state4, 'e3')

In [22]:
direct_effects

{A, C}

In [23]:
indirect_effects

{B}

## Ensuring Event

In [24]:
get_ensuring_event(scenario_path, A)

'e3'

In [25]:
get_ensuring_event(scenario_path, B)

'e3'

In [26]:
get_ensuring_event(scenario_path, C)

'e3'

In [27]:
get_ensuring_event(scenario_path, D)

'e2'

In [28]:
get_ensuring_event(scenario_path, E)

'e1'

In [29]:
get_ensuring_event(scenario_path, F)

## Second Causal Explanation

In [30]:
supporting_literals, initial_literals = get_second_causal_explanation(scenario_path, state4, effects=direct_effects | indirect_effects)

In [31]:
supporting_literals

{('e1', E), ('e2', D)}

In [32]:
initial_literals

{F}

## Third Causal Explanation

In [33]:
supporting_events, uncaused_literals = get_third_causal_explanation(AD, scenario_path)

In [34]:
supporting_events

{('e1', E)}

In [35]:
uncaused_literals

{¬A, ¬C, F}